## Processamento inicial da base de dados MIMIC-IV

O objetivo desse notebook é realizar um processamento das tabelas originais do MIMIC-IV com objetivo de gerar tabelas intermediarias contendo apenas os dados utilizados na análise exploratória

### Obtenção de dados de CK (CPK)

Inicialmente filtramos a tabela `chartevents.csv` que possui diversas informações sobre eventos do hospital inclusive sobre o exame de CK (CPK), identificado pela coluna `itemid` igual a `225634` (pode ser verificado utilizando a tabela `d_items.csv`). Assim, filtramos apenas esses exames. O resultado é salvo na tabela `ck_events.csv`

Como a tabela `chartevents.csv` é muito grande (30gb) ela foi processada em pedaços (chunks).


In [1]:
path_chartevents = "/media/salathiel/HD/mimic/chartevents.csv"
outfilename = "../data/interim/ck_events_2.csv"

In [2]:
import pandas as pd

ck_cpk_id = 225634
chunksize = 10 ** 7  #quantidade de linhas a serem lidas de uma vez

df_final = None
with pd.read_csv(path_chartevents, chunksize=chunksize) as reader:
    for chunk in reader:
        chunk = chunk.loc[chunk['itemid'] == ck_cpk_id]
        if len(chunk) > 0:
            if df_final is None:
                df_final = chunk
            else:
                df_final = pd.concat([df_final,chunk])

print("Saving processed table in:", outfilename)
df_final.to_csv(outfilename)

KeyboardInterrupt: 

In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(outfilename)
print(df.head())
print("LEN:", len(df))

#### Informaçoes dos pacientes que fizeram exames de CK (CPK)

A tabela gerada (`ck_events.csv`) possui o identificador do paciente que realizou o exame na coluna `subject_id`. Utilizando essa informação podemos pegar as informações desses pacientes. As informações dos pacientes do MIMIC-IV estão disponíveis na tabela `omr.csv`, que possui informações sobre peso, altura, IMC e pressão arterial.
O código abaixo obtêm as informações de todos os pacientes que fizeram algum exame CK (CPK), e salva na tabela `pacients_with_ck.csv`

In [ ]:
glucose_events_path = "../data/interim/glucose_events.csv"
output_pacients_with_ck = "../data/interim/pacients_with_ck.csv"

In [ ]:
df_g = pd.read_csv(glucose_events_path)
all_pacients = df_g['subject_id']

df_pacients = pd.read_csv(pacient_info_path)
df_pacients_with_ck = df_pacients[df_pacients['subject_id'].isin(all_pacients)]

print("Saving processed table in:", output_pacients_with_ck)
df_pacients_with_ck.to_csv(output_pacients_with_ck)

In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(output_pacients_with_ck)
print(df.head())
print("LEN:", len(df))

#### Informações sobre aplicações de insulina em pacientes com CK (CPK)
Além das informações obtidas no processamento acima, também estamos interessados em verificar se esses pacientes receberam doses de insulina.
A tabela `inputevents.csv` possui informações sobre todos os eventos de entradas de fluido (incluindo insulina). Observando novamente a tabela `d_items.csv`, a insulina aparece com `itemid` com os números: 223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619. Esses números se referem as insulinas descritas como:  Insulin - 70/30, Insulin - Regular, Insulin - NPH, Insulin - Glargine, Insulin - Humalog 75/25, Insulin - Humalog,Insulin - Novolog, Insulin - U500

A partir dessa tabela é gerada uma nova tabela com o nome de `insulin_inputs_in_ck_pacients.csv` com todos os eventos de aplicação de insulina em pacientes que fizeram algum exame de CK (CPK).

Novamente essa tabela (`inputevents.csv`) é muito grande (2,3GB) assim também foi processada em pedaços (chunks).

In [ ]:
input_events_path = "../data/external/inputevents.csv"
insulin_inputs_output = "../data/interim/insulin_inputs_in_ck_pacients.csv"
insulin_idxs = [223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619]

In [ ]:
chunksize = 10 ** 6

df_ck = pd.read_csv(ck_events_path)
all_pacients = df_ck['subject_id']

df_final = None
with pd.read_csv(input_events_path, chunksize=chunksize) as reader:
    for chunk in reader:
        df_pacients_with_ck = chunk[chunk['subject_id'].isin(all_pacients)]
        df_insulin_inputs = df_pacients_with_ck[df_pacients_with_ck['itemid'].isin(insulin_idxs)]
        if len(chunk) > 0:
            if df_final is None:
                df_final = df_insulin_inputs
            else:
                df_final = pd.concat([df_final,df_insulin_inputs])


print("Saving processed table in:", insulin_inputs_output)
df_final.to_csv(insulin_inputs_output)

In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(insulin_inputs_output)
print(df.head())
print("LEN:", len(df))

### Obtenção de dados de glicose

Filtramos novamente a tabela `chartevents.csv` identificando pela coluna `itemid`. Observando a tabela `d_items.csv`, a glicose aparece de diversas formas. Foram selecionados somente IDs que aparecem na tabela `chartevents.cs` e, dentre esses, somente os IDs 220621, 225664, 226537, 228388 que se referem a: Glucose (serum), Glucose finger stick (range 70-100), Glucose (whole blood), Glucose (whole blood) (soft). Não foram selecionados os IDS referentes ao sistema de escore Apache IV pois ele não é o foco do estudo. Também foi omitido o ID da categoria Glucose Control - Prophy por ser um parâmetro textual relacionado à profilaxia. Assim, filtramos apenas medições de glicose, salvando o resultado na tabela `glucose_events.csv`. Novamente, processamos `chartevents.csv` em chunks.

In [1]:
import pandas as pd
path_chartevents = "/media/salathiel/HD/mimic/chartevents.csv"
outfilename = "../data/interim/glucose_events.csv"
glucose_idxs = [220621, 225664, 226537, 228388]
chunksize = 10 ** 7  #quantidade de linhas a serem lidas de uma vez

df_final = None
with pd.read_csv(path_chartevents, chunksize=chunksize) as reader:
    for chunk in reader:
        chunk = chunk.loc[chunk['itemid'].isin(glucose_idxs)]
        if len(chunk) > 0:
            if df_final is None:
                df_final = chunk
            else:
                df_final = pd.concat([df_final,chunk])


print("Saving processed table in:", outfilename)
df_final.to_csv(outfilename)

Saving processed table in: ../data/interim/glucose_events.csv


In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(outfilename)
print(df.head())
print("LEN:", len(df))

#### Informaçoes dos pacientes que fizeram exames de glicose

A tabela gerada (`glucose_events.csv`) possui o identificador do paciente que realizou o exame na coluna `subject_id`. Utilizando essa informação podemos pegar as informações desses pacientes. As informações dos pacientes do MIMIC-IV estão disponíveis na tabela `omr.csv`, que possui informações sobre peso, altura, IMC e pressão arterial.

O código abaixo obtêm as informações de todos os pacientes que fizeram algum exame de glicose, e salva na tabela `patients_with_glucose.csv`

In [2]:
patient_info_path = "/media/salathiel/HD/mimic/omr.csv"
glucose_events_path = "../data/interim/glucose_events.csv"
output_patients_with_glucose = "../data/interim/pacients_with_glucose.csv"

In [3]:
df_glucose = pd.read_csv(glucose_events_path)
all_patients = df_glucose['subject_id']

df_patients = pd.read_csv(patient_info_path)
df_patients_with_glucose = df_patients[df_patients['subject_id'].isin(all_patients)]

print("Saving processed table in:", output_patients_with_glucose)
df_patients_with_glucose.to_csv(output_patients_with_glucose)

Saving processed table in: ../data/interim/pacients_with_glucose.csv


In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(output_patients_with_glucose)
print(df.head())
print("LEN:", len(df))

#### Informações sobre aplicações de insulina em pacientes com glicose

Vamos novamente analisar pacientes que receberam aplicação de insulina, mas agora considerando pacientes que fizeram exame de glicose ao invés de CK (CPK).
Para tanto, vamos usar os mesmos IDs analisados anteriormente.

A partir dessa tabela é gerada uma nova tabela com o nome de `insulin_inputs_in_glucose_patients.csv` com todos os eventos de aplicação de insulina em pacientes que fizeram algum exame de glicose selecionado anteriormente.

Novamente essa tabela (`inputevents.csv`) é muito grande (2,3GB) assim também foi processada em pedaços (chunks).

In [4]:
input_events_path = "/media/salathiel/HD/mimic/inputevents.csv"
insulin_inputs_output = "../data/interim/insulin_inputs_in_glucose_pacients.csv"

insulin_idxs = [223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619]

In [5]:
chunksize = 10 ** 6

df_glucose = pd.read_csv(glucose_events_path)
all_patients = df_glucose['subject_id']

df_final = None
with pd.read_csv(input_events_path, chunksize=chunksize) as reader:
    for chunk in reader:
        df_patients_with_glucose = chunk[chunk['subject_id'].isin(all_patients)]
        df_insulin_inputs = df_patients_with_glucose[df_patients_with_glucose['itemid'].isin(insulin_idxs)]
        if len(chunk) > 0:
            if df_final is None:
                df_final = df_insulin_inputs
            else:
                df_final = pd.concat([df_final,df_insulin_inputs])


print("Saving processed table in:", insulin_inputs_output)
df_final.to_csv(insulin_inputs_output)

Saving processed table in: ../data/interim/insulin_inputs_in_glucose_pacients.csv


In [ ]:
#visualizando a tabela gerada
df = pd.read_csv(insulin_inputs_output)
print(df.head())
print("LEN:", len(df))